# Calculate features for downloaded light curves

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
import sqlalchemy as db

In [ ]:
widths = (8,7,4,13,43)
header_pd = pd.read_fwf('../databases/Labeled_data.txt', widths = widths,skiprows=7, nrows=27)
labeled_data = pd.read_csv('../databases/Labeled_data.txt', header=None, delim_whitespace=True, skiprows=37) # extract data
labeled_data.columns = header_pd.iloc[:,3]
labeled_data.head()

In [ ]:
label = labeled_data.Type.unique()
label = np.delete(label, np.where(label == 'CEPII')) # CEPII has very few example so we ignore it
label = np.delete(label, np.where(label == 'CEP')) # CEP has very few example so we ignore it

print(label)

In [ ]:
RA = []
Dec = []
SourceID = []
Type = []
ID = []
g_mean = []
g_wmean = [] # weighted mean
g_MAD = []
g_IQR = []
g_f60 = []
g_f70 = []
g_f80 = []
g_f90 = []
g_skew = []
g_kurtosis = []
g_welsh_K = []
g_welsh_J = []

r_mean = []
r_wmean = [] # weighted mean
r_MAD = []
r_IQR = []
r_f60 = []
r_f70 = []
r_f80 = []
r_f90 = []
r_skew = []
r_kurtosis = []
r_welsh_K = []
r_welsh_J = []

In [ ]:
def weighted_mean_del(mag,mag_err):
    mag2 = (mag_err*mag_err) # mag err square
    mag2_inv = 1/mag2.values; # take inverse of the values
    w = pd.Series(mag2_inv) # covert it back to s series
    sw = w.sum() # sum of weights
    wmag = mag*w # multiply magnitude with weights
    wmean = wmag.sum()/sw # weighted mean
    return wmean

In [ ]:
def weighted_mean(mag,e_mag):
    w = 1/(e_mag*e_mag)
    sw = np.sum(w)
    wmag = w*mag
    wmean = np.sum(wmag)/sw
    return wmean

In [ ]:
# welsh J, K statistics
def welsh_staton(mag_series,wmean):
    N = len(mag_series)
    d_i = N/(N-1)*(mag_series - wmean) # replace mean by weighted mean
    d_i1 = d_i.shift(periods=-1)
    d_i1.fillna(0, inplace = True)
    Pi = d_i*d_i1
    Pi_val = Pi.values
    Psign = np.sign(Pi_val)
    Jval = Psign*np.sqrt(np.abs(Pi_val))
    J = np.sum(Jval) 
    K1 = abs(d_i.values)/N
    K2 = np.sqrt(1/N*np.sum(d_i.values*d_i.values))
    K = np.sum(K1*K2)
    return J, K 

In [ ]:
engine = db.create_engine('sqlite:///../databases/save_LC_1000each.db', echo=False)
sqlite_connection = engine.connect()
metadata = db.MetaData()
metadata.reflect(bind=engine)
LC={}
table_list = metadata.sorted_tables


In [ ]:
i = 0
for table in table_list:
    data = db.Table(table, metadata, autoload=True, autoload_with=engine)
    query = db.select([data])
    ResultProxy = sqlite_connection.execute(query)
    ResultSet = ResultProxy.fetchall()
    lc = pd.DataFrame(ResultSet)
    if len(lc) >1:
        lc.columns = ResultSet[0].keys()
        RA.append(lc.RAdeg[1])
        Dec.append(lc.DEdeg[1])
        SourceID.append(lc.SourceID[1])
        ID.append(lc.ID[1])
        Type.append(lc.Type[1])
        
        dfg = lc.loc[lc["band"] == "g"]
        dfr = lc.loc[lc["band"] == "r"]
        if len(dfg) > 1:
            N = len(dfg)
    #         wmean_temp = np.average(dfg.mag, weights = dfg.e_mag*dfg.e_mag)
            wmean_temp = weighted_mean(dfg.mag.values,dfg.e_mag.values)
            K_temp, J_temp =  welsh_staton(dfg.mag, wmean_temp)
            g_mean.append(dfg.mag.mean())
            g_wmean.append(wmean_temp) 
            deviation = abs(dfg.mag - dfg.mag.median())
            g_MAD.append(deviation.median())
            g_IQR.append(dfg.mag.quantile(0.75) - dfg.mag.quantile(0.25))
            g_f60.append(dfg.mag.quantile(0.80) - dfg.mag.quantile(0.2))
            g_f70.append(dfg.mag.quantile(0.85) - dfg.mag.quantile(0.15))
            g_f80.append(dfg.mag.quantile(0.9) - dfg.mag.quantile(0.10))
            g_f90.append(dfg.mag.quantile(0.95) - dfg.mag.quantile(0.05))
            g_skew.append(dfg.mag.skew())
            g_kurtosis.append(dfg.mag.kurtosis())
            g_welsh_J.append(J_temp)
            g_welsh_K.append(K_temp)
        else:
            g_mean.append(np.NaN)
            g_wmean.append(np.NaN) 
            g_MAD.append(np.NaN)
            g_IQR.append(np.NaN)
            g_f60.append(np.NaN)
            g_f70.append(np.NaN)
            g_f80.append(np.NaN)
            g_f90.append(np.NaN)
            g_skew.append(np.NaN)
            g_kurtosis.append(np.NaN)
            g_welsh_J.append(np.NaN)
            g_welsh_K.append(np.NaN)
                
        if len(dfr) >1:
            N = len(dfr)
            wmean_temp = weighted_mean(dfr.mag.values,dfr.e_mag.values)
    #         wmean_temp = np.average(dfr.mag, weights = dfr.e_mag*dfr.e_mag)
            K_temp, J_temp =  welsh_staton(dfr.mag, wmean_temp )
            r_mean.append(dfr.mag.mean())
            r_wmean.append(wmean_temp) 
            deviation = abs(dfr.mag - dfr.mag.median())
            r_MAD.append(deviation.median())
            r_IQR.append(dfr.mag.quantile(0.75) - dfr.mag.quantile(0.25))
            r_f60.append(dfr.mag.quantile(0.80) - dfr.mag.quantile(0.2))
            r_f70.append(dfr.mag.quantile(0.85) - dfr.mag.quantile(0.15))
            r_f80.append(dfr.mag.quantile(0.9) - dfr.mag.quantile(0.10))
            r_f90.append(dfr.mag.quantile(0.95) - dfr.mag.quantile(0.05))
            r_skew.append(dfr.mag.skew())
            r_kurtosis.append(dfr.mag.kurtosis())
            r_welsh_J.append(J_temp)
            r_welsh_K.append(K_temp)
        else:
            r_mean.append(np.NaN)
            r_wmean.append(np.NaN) 
            r_MAD.append(np.NaN)
            r_IQR.append(np.NaN)
            r_f60.append(np.NaN)
            r_f70.append(np.NaN)
            r_f80.append(np.NaN)
            r_f90.append(np.NaN)
            r_skew.append(np.NaN)
            r_kurtosis.append(np.NaN)
            r_welsh_J.append(np.NaN)
            r_welsh_K.append(np.NaN)

    else:
        RA.append(np.NaN)
        Dec.append(np.NaN)
        SourceID.append(np.NaN) 
        ID.append(np.NaN)
        Type.append(np.NaN)
        g_mean.append(np.NaN)
        g_wmean.append(np.NaN) 
        g_MAD.append(np.NaN)
        g_IQR.append(np.NaN)
        g_f60.append(np.NaN)
        g_f70.append(np.NaN)
        g_f80.append(np.NaN)
        g_f90.append(np.NaN)
        g_skew.append(np.NaN)
        g_kurtosis.append(np.NaN)
        g_welsh_J.append(np.NaN)
        g_welsh_K.append(np.NaN)
        r_mean.append(np.NaN)
        r_wmean.append(np.NaN) 
        r_MAD.append(np.NaN)
        r_IQR.append(np.NaN) 
        r_f60.append(np.NaN)
        r_f70.append(np.NaN)
        r_f80.append(np.NaN)
        r_f90.append(np.NaN)
        r_skew.append(np.NaN)
        r_kurtosis.append(np.NaN)
        r_welsh_J.append(np.NaN)
        r_welsh_K.append(np.NaN)
    print(i)
    i = i+1

In [ ]:
len(lc)

In [ ]:
# del features
features = pd.DataFrame()
N = 17998
features['sourceid'] = SourceID[0:N]
features['ID'] = ID[0:N]
# features['ID'] = labeled_data_sampled.ID.values[0:N]
features['RAdeg'] = RA[0:N]
features['DEdeg'] = Dec[0:N]

# g filter data
features['g_mean'] = g_mean[0:N]
features['g_wmean'] = g_wmean[0:N]
features['g_MAD'] = g_MAD[0:N]
features['g_IQR'] = g_IQR[0:N]
features['g_f60'] = g_f60[0:N]
features['g_f70'] = g_f70[0:N]
features['g_f80'] = g_f80[0:N]
features['g_f90'] = g_f90[0:N]
features['g_skew'] = g_skew[0:N]
features['g_kurtosis'] = g_kurtosis[0:N]
features['g_welsh_J'] = g_welsh_J[0:N]
features['g_welsh_K'] = g_welsh_K[0:N]

# r filter data
features['r_mean'] = r_mean[0:N]
features['r_wmean'] = r_wmean[0:N]
features['r_MAD'] = r_MAD[0:N]
features['r_IQR'] = r_IQR[0:N]
features['r_f60'] = r_f60[0:N]
features['r_f70'] = r_f70[0:N]
features['r_f80'] = r_f80[0:N]
features['r_f90'] = r_f90[0:N]
features['r_skew'] = r_skew[0:N]
features['r_kurtosis'] = r_kurtosis[0:N]
features['r_welsh_J'] = r_welsh_J[0:N]
features['r_welsh_K'] = r_welsh_K[0:N]
features['Type'] = Type[0:N]

# features['label'] = label[0:N]
# features.to_csv('../features/features10000.csv',index=False)

In [ ]:
features.tail(100)

In [ ]:
features_pd = pd.read_csv('../features/features20000.csv',header='infer')

In [ ]:
# features_pd['Type'] = 'EW'
features_pd.info()

In [ ]:
for k in range(0,9):
    for i in range(0,2000):
        features_pd.loc[(i+k*1999),'Type'] = label[k]
    

In [ ]:
features_pd.head()

In [ ]:
features_pd.dropna(inplace=True)

In [ ]:
features_pd.info()

In [ ]:
import seaborn as sns
sns.histplot(data=features_pd, x=features_pd.Type)